In [1]:
#Importacion de librerias
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')

In [2]:
#Lectura de archivos CSV
train = pd.read_csv('../../DATA_TP2/train.csv')

In [3]:
#Nuevo dataframe
trainProcesado = pd.DataFrame()

#Copiamos la columna con el id
trainProcesado['id'] = train['id']
trainProcesado['idzona'] = train['idzona']

#Manejo de NaN, columnas numericas
train['garages'].fillna(0, inplace=True)
train['banos'].fillna(1, inplace=True)

promedio_antiguedad_x_ciudad = train.groupby(['ciudad','idzona'])['antiguedad'].transform('mean')
train['antiguedad'].fillna(promedio_antiguedad_x_ciudad, inplace=True)
train['antiguedad'].fillna(train['antiguedad'].mean(), inplace=True)

promedio_habitaciones_x_ciudad = train.groupby(['ciudad','idzona'])['habitaciones'].transform('mean')
train['habitaciones'].fillna(promedio_habitaciones_x_ciudad, inplace=True)
train['habitaciones'].fillna(train['habitaciones'].mean(), inplace=True)

promedio_metroscubierto_x_ciudad = train.groupby(['ciudad','idzona'])['metroscubiertos'].transform('mean')
train['metroscubiertos'].fillna(promedio_metroscubierto_x_ciudad, inplace=True)
train['metroscubiertos'].fillna(train['metroscubiertos'].mean(), inplace=True)

promedio_metrostotales_x_ciudad = train.groupby(['ciudad','idzona'])['metrostotales'].transform('mean')
train['metrostotales'].fillna(promedio_metrostotales_x_ciudad, inplace=True)
train['metrostotales'].fillna(train['metrostotales'].mean(), inplace=True)

#Manejo de NaN, columnas catedoricas
train['ciudad'] = train['ciudad'].fillna('Desconocida')
train['provincia'] = train['provincia'].fillna('Desconocida')
train['tipodepropiedad'] = train['tipodepropiedad'].fillna('Desconocido')

train['ciudad'] = train['ciudad'].str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
train['provincia'] = train['provincia'].str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
train['tipodepropiedad'] = train['tipodepropiedad'].str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

train['ciudad'] = train['ciudad'].astype('category')
train['provincia'] = train['provincia'].astype('category')
train['tipodepropiedad'] = train['tipodepropiedad'].astype('category')

train['ciudad'] = train['ciudad'].astype('category')
trainProcesado['provincia'] = train['provincia'].astype('category')
trainProcesado['tipodepropiedad'] = train['tipodepropiedad'].astype('category')

#Asignacion de tipos de datos
trainProcesado['antiguedad'] = train['antiguedad'].astype('int64')
trainProcesado['habitaciones'] = train['habitaciones'].astype('int64')
trainProcesado['garages'] = train['garages'].astype('int64')
trainProcesado['banos'] = train['banos'].astype('int64')
trainProcesado['metroscubiertos'] = train['metroscubiertos'].astype('int64')
trainProcesado['metrostotales'] = train['metrostotales'].astype('int64')

trainProcesado['gimnasio'] = train['gimnasio'].astype('int64')
trainProcesado['usosmultiples'] = train['usosmultiples'].astype('int64')
trainProcesado['piscina'] = train['piscina'].astype('int64')
trainProcesado['escuelascercanas'] = train['escuelascercanas'].astype('int64')
trainProcesado['escuelascercanas'] = train['escuelascercanas'].astype('int64')
trainProcesado['centroscomercialescercanos'] = train['centroscomercialescercanos'].astype('int64')

#Solo nos quedamos con el anio de la fecha
trainProcesado['anio'] = pd.to_datetime(train['fecha'], infer_datetime_format=True).dt.year

trainProcesado['2012'] = 0
trainProcesado['2013'] = 0
trainProcesado['2014'] = 0
trainProcesado['2015'] = 0
trainProcesado['2016'] = 0

trainProcesado.loc[train['fecha'].str.contains('2012', na=False), '2012'] = 1
trainProcesado.loc[train['fecha'].str.contains('2013', na=False), '2013'] = 1
trainProcesado.loc[train['fecha'].str.contains('2014', na=False), '2014'] = 1
trainProcesado.loc[train['fecha'].str.contains('2015', na=False), '2015'] = 1
trainProcesado.loc[train['fecha'].str.contains('2016', na=False), '2016'] = 1

#Precio
trainProcesado['precio'] = train['precio']

In [4]:
trainProcesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 22 columns):
id                            240000 non-null int64
idzona                        211379 non-null float64
provincia                     240000 non-null category
tipodepropiedad               240000 non-null category
antiguedad                    240000 non-null int64
habitaciones                  240000 non-null int64
garages                       240000 non-null int64
banos                         240000 non-null int64
metroscubiertos               240000 non-null int64
metrostotales                 240000 non-null int64
gimnasio                      240000 non-null int64
usosmultiples                 240000 non-null int64
piscina                       240000 non-null int64
escuelascercanas              240000 non-null int64
centroscomercialescercanos    240000 non-null int64
anio                          240000 non-null int64
2012                          240000 non-null int

In [5]:
trainProcesado['precio'] = trainProcesado['precio'].astype(int)

In [6]:
(trainProcesado[trainProcesado['precio'] > 11000000])['id'].count() # 12525000 # 310000

1410

In [7]:
## https://es.inflation.eu/tasas-de-inflacion/mexico/inflacion-historica/ipc-inflacion-mexico.aspx

## 2012  IPC México 2012 	3,57 %
## 2013  IPC México 2013 	3,97 %
## 2014  IPC México 2014 	4,08 %
## 2015  IPC México 2015 	2,13 %
## 2016  IPC México 2016 	3,36 %

trainProcesado['precio_2012'] = 0
trainProcesado['precio_2013'] = 0
trainProcesado['precio_2014'] = 0
trainProcesado['precio_2015'] = 0
trainProcesado['precio_2016'] = 0

trainProcesado.loc[trainProcesado.anio == 2012, 'precio_2012'] = trainProcesado['precio']
trainProcesado.loc[trainProcesado.anio == 2012, 'precio_2013'] = trainProcesado['precio_2012'] * 1.0357
trainProcesado.loc[trainProcesado.anio == 2012, 'precio_2014'] = trainProcesado['precio_2013'] * 1.0397
trainProcesado.loc[trainProcesado.anio == 2012, 'precio_2015'] = trainProcesado['precio_2014'] * 1.0408
trainProcesado.loc[trainProcesado.anio == 2012, 'precio_2016'] = trainProcesado['precio_2015'] * 1.0213

trainProcesado.loc[trainProcesado.anio == 2013, 'precio_2013'] = trainProcesado['precio']
trainProcesado.loc[trainProcesado.anio == 2013, 'precio_2012'] = trainProcesado['precio_2013'] * (1 - 0.0357)
trainProcesado.loc[trainProcesado.anio == 2013, 'precio_2014'] = trainProcesado['precio_2013'] * 1.0397
trainProcesado.loc[trainProcesado.anio == 2013, 'precio_2015'] = trainProcesado['precio_2014'] * 1.0408
trainProcesado.loc[trainProcesado.anio == 2013, 'precio_2016'] = trainProcesado['precio_2015'] * 1.0213

trainProcesado.loc[trainProcesado.anio == 2014, 'precio_2014'] = trainProcesado['precio']
trainProcesado.loc[trainProcesado.anio == 2014, 'precio_2013'] = trainProcesado['precio_2014'] * (1 - 0.0397)
trainProcesado.loc[trainProcesado.anio == 2014, 'precio_2012'] = trainProcesado['precio_2013'] * (1 - 0.0357)
trainProcesado.loc[trainProcesado.anio == 2014, 'precio_2015'] = trainProcesado['precio_2014'] * 1.0408
trainProcesado.loc[trainProcesado.anio == 2014, 'precio_2016'] = trainProcesado['precio_2015'] * 1.0213

trainProcesado.loc[trainProcesado.anio == 2015, 'precio_2015'] = trainProcesado['precio']
trainProcesado.loc[trainProcesado.anio == 2015, 'precio_2014'] = trainProcesado['precio_2015'] * (1 - 0.0408)
trainProcesado.loc[trainProcesado.anio == 2015, 'precio_2013'] = trainProcesado['precio_2014'] * (1 - 0.0397)
trainProcesado.loc[trainProcesado.anio == 2015, 'precio_2012'] = trainProcesado['precio_2013'] * (1 - 0.0357)
trainProcesado.loc[trainProcesado.anio == 2015, 'precio_2016'] = trainProcesado['precio_2015'] * 1.0213

trainProcesado.loc[trainProcesado.anio == 2016, 'precio_2016'] = trainProcesado['precio']
trainProcesado.loc[trainProcesado.anio == 2016, 'precio_2015'] = trainProcesado['precio_2016'] * (1 - 0.0213)
trainProcesado.loc[trainProcesado.anio == 2016, 'precio_2014'] = trainProcesado['precio_2015'] * (1 - 0.0408)
trainProcesado.loc[trainProcesado.anio == 2016, 'precio_2013'] = trainProcesado['precio_2014'] * (1 - 0.0397)
trainProcesado.loc[trainProcesado.anio == 2016, 'precio_2012'] = trainProcesado['precio_2013'] * (1 - 0.0357)

In [8]:
dummies = pd.get_dummies(trainProcesado['provincia'], drop_first=False)
trainProcesado = pd.concat([trainProcesado, dummies], axis=1)

dummies = pd.get_dummies(trainProcesado['tipodepropiedad'], drop_first=False)
trainProcesado = pd.concat([trainProcesado, dummies], axis=1)

In [9]:
#AGRUPAMIENTO POR ANTIGUEDAD (BANDAS)
trainProcesado['antiguedad_agrupada'] = 'antiguedad_agrupada_0'

trainProcesado.loc[trainProcesado.antiguedad <= 5, 'antiguedad_agrupada'] = 'antiguedad_agrupada_0'
trainProcesado.loc[trainProcesado.antiguedad > 95, 'antiguedad_agrupada'] = 'antiguedad_agrupada_100'
trainProcesado.loc[((trainProcesado.antiguedad > 5) & (trainProcesado.antiguedad <= 15)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_10'
trainProcesado.loc[((trainProcesado.antiguedad > 15) & (trainProcesado.antiguedad <= 25)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_20'
trainProcesado.loc[((trainProcesado.antiguedad > 25) & (trainProcesado.antiguedad <= 35)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_30'
trainProcesado.loc[((trainProcesado.antiguedad > 35) & (trainProcesado.antiguedad <= 45)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_40'
trainProcesado.loc[((trainProcesado.antiguedad > 45) & (trainProcesado.antiguedad <= 55)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_50'
trainProcesado.loc[((trainProcesado.antiguedad > 55) & (trainProcesado.antiguedad <= 65)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_60'
trainProcesado.loc[((trainProcesado.antiguedad > 65) & (trainProcesado.antiguedad <= 75)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_70'
trainProcesado.loc[((trainProcesado.antiguedad > 75) & (trainProcesado.antiguedad <= 85)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_80'
trainProcesado.loc[((trainProcesado.antiguedad > 85) & (trainProcesado.antiguedad <= 95)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_90'

trainProcesado.drop('antiguedad', 1, inplace = True)

# AGRUPAMIENTO POR METROS CUBIENTOS (BANDAS)
trainProcesado['metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_0'

trainProcesado.loc[trainProcesado.metroscubiertos <= 5, 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_0'
trainProcesado.loc[trainProcesado.metroscubiertos > 805, 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_1000'
trainProcesado.loc[((trainProcesado.metroscubiertos > 5) & (trainProcesado.metroscubiertos <= 15)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_10'
trainProcesado.loc[((trainProcesado.metroscubiertos > 15) & (trainProcesado.metroscubiertos <= 25)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_20'
trainProcesado.loc[((trainProcesado.metroscubiertos > 25) & (trainProcesado.metroscubiertos <= 35)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_30'
trainProcesado.loc[((trainProcesado.metroscubiertos > 35) & (trainProcesado.metroscubiertos <= 45)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_40'
trainProcesado.loc[((trainProcesado.metroscubiertos > 45) & (trainProcesado.metroscubiertos <= 55)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_50'
trainProcesado.loc[((trainProcesado.metroscubiertos > 55) & (trainProcesado.metroscubiertos <= 65)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_60'
trainProcesado.loc[((trainProcesado.metroscubiertos > 65) & (trainProcesado.metroscubiertos <= 75)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_70'
trainProcesado.loc[((trainProcesado.metroscubiertos > 75) & (trainProcesado.metroscubiertos <= 85)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_80'
trainProcesado.loc[((trainProcesado.metroscubiertos > 85) & (trainProcesado.metroscubiertos <= 95)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_90'
trainProcesado.loc[((trainProcesado.metroscubiertos > 95) & (trainProcesado.metroscubiertos <= 105)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_100'
trainProcesado.loc[((trainProcesado.metroscubiertos > 105) & (trainProcesado.metroscubiertos <= 115)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_110'
trainProcesado.loc[((trainProcesado.metroscubiertos > 115) & (trainProcesado.metroscubiertos <= 130)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_125'
trainProcesado.loc[((trainProcesado.metroscubiertos > 130) & (trainProcesado.metroscubiertos <= 155)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_150'
trainProcesado.loc[((trainProcesado.metroscubiertos > 155) & (trainProcesado.metroscubiertos <= 205)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_200'
trainProcesado.loc[((trainProcesado.metroscubiertos > 205) & (trainProcesado.metroscubiertos <= 255)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_250'
trainProcesado.loc[((trainProcesado.metroscubiertos > 255) & (trainProcesado.metroscubiertos <= 305)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_300'
trainProcesado.loc[((trainProcesado.metroscubiertos > 305) & (trainProcesado.metroscubiertos <= 355)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_350'
trainProcesado.loc[((trainProcesado.metroscubiertos > 355) & (trainProcesado.metroscubiertos <= 405)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_400'
trainProcesado.loc[((trainProcesado.metroscubiertos > 405) & (trainProcesado.metroscubiertos <= 455)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_450'
trainProcesado.loc[((trainProcesado.metroscubiertos > 455) & (trainProcesado.metroscubiertos <= 500)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_500'
trainProcesado.loc[((trainProcesado.metroscubiertos > 505) & (trainProcesado.metroscubiertos <= 555)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_550'
trainProcesado.loc[((trainProcesado.metroscubiertos > 555) & (trainProcesado.metroscubiertos <= 600)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_600'
trainProcesado.loc[((trainProcesado.metroscubiertos > 605) & (trainProcesado.metroscubiertos <= 655)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_650'
trainProcesado.loc[((trainProcesado.metroscubiertos > 655) & (trainProcesado.metroscubiertos <= 700)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_700'
trainProcesado.loc[((trainProcesado.metroscubiertos > 705) & (trainProcesado.metroscubiertos <= 755)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_750'
trainProcesado.loc[((trainProcesado.metroscubiertos > 755) & (trainProcesado.metroscubiertos <= 805)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_800'

trainProcesado.drop('metroscubiertos', 1, inplace = True)

# AGRUPAMIENTO POR METROS TOTALES (BANDAS)
trainProcesado['metrostotales_agrupada'] = 'metrostotales_agrupada_0'

trainProcesado.loc[trainProcesado.metrostotales <= 5, 'metrostotales_agrupada'] = 'metrostotales_agrupada_0'
trainProcesado.loc[trainProcesado.metrostotales > 805, 'metrostotales_agrupada'] = 'metrostotales_agrupada_1000'
trainProcesado.loc[((trainProcesado.metrostotales > 5) & (trainProcesado.metrostotales <= 15)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_10'
trainProcesado.loc[((trainProcesado.metrostotales > 15) & (trainProcesado.metrostotales <= 25)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_20'
trainProcesado.loc[((trainProcesado.metrostotales > 25) & (trainProcesado.metrostotales <= 35)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_30'
trainProcesado.loc[((trainProcesado.metrostotales > 35) & (trainProcesado.metrostotales <= 45)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_40'
trainProcesado.loc[((trainProcesado.metrostotales > 45) & (trainProcesado.metrostotales <= 55)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_50'
trainProcesado.loc[((trainProcesado.metrostotales > 55) & (trainProcesado.metrostotales <= 65)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_60'
trainProcesado.loc[((trainProcesado.metrostotales > 65) & (trainProcesado.metrostotales <= 75)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_70'
trainProcesado.loc[((trainProcesado.metrostotales > 75) & (trainProcesado.metrostotales <= 85)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_80'
trainProcesado.loc[((trainProcesado.metrostotales > 85) & (trainProcesado.metrostotales <= 95)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_90'
trainProcesado.loc[((trainProcesado.metrostotales > 95) & (trainProcesado.metrostotales <= 105)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_100'
trainProcesado.loc[((trainProcesado.metrostotales > 105) & (trainProcesado.metrostotales <= 115)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_110'
trainProcesado.loc[((trainProcesado.metrostotales > 115) & (trainProcesado.metrostotales <= 130)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_125'
trainProcesado.loc[((trainProcesado.metrostotales > 130) & (trainProcesado.metrostotales <= 155)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_150'
trainProcesado.loc[((trainProcesado.metrostotales > 155) & (trainProcesado.metrostotales <= 205)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_200'
trainProcesado.loc[((trainProcesado.metrostotales > 205) & (trainProcesado.metrostotales <= 255)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_250'
trainProcesado.loc[((trainProcesado.metrostotales > 255) & (trainProcesado.metrostotales <= 305)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_300'
trainProcesado.loc[((trainProcesado.metrostotales > 305) & (trainProcesado.metrostotales <= 355)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_350'
trainProcesado.loc[((trainProcesado.metrostotales > 355) & (trainProcesado.metrostotales <= 405)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_400'
trainProcesado.loc[((trainProcesado.metrostotales > 405) & (trainProcesado.metrostotales <= 455)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_450'
trainProcesado.loc[((trainProcesado.metrostotales > 455) & (trainProcesado.metrostotales <= 500)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_500'
trainProcesado.loc[((trainProcesado.metrostotales > 505) & (trainProcesado.metrostotales <= 555)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_550'
trainProcesado.loc[((trainProcesado.metrostotales > 555) & (trainProcesado.metrostotales <= 600)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_600'
trainProcesado.loc[((trainProcesado.metrostotales > 605) & (trainProcesado.metrostotales <= 655)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_650'
trainProcesado.loc[((trainProcesado.metrostotales > 655) & (trainProcesado.metrostotales <= 700)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_700'
trainProcesado.loc[((trainProcesado.metrostotales > 705) & (trainProcesado.metrostotales <= 755)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_750'
trainProcesado.loc[((trainProcesado.metrostotales > 755) & (trainProcesado.metrostotales <= 805)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_800'

trainProcesado.drop('metrostotales', 1, inplace = True)

In [10]:
dummies = pd.get_dummies(trainProcesado['antiguedad_agrupada'], drop_first=False)
trainProcesado = pd.concat([trainProcesado, dummies], axis=1)

dummies = pd.get_dummies(trainProcesado['metroscubiertos_agrupada'], drop_first=False)
trainProcesado = pd.concat([trainProcesado, dummies], axis=1)

dummies = pd.get_dummies(trainProcesado['metrostotales_agrupada'], drop_first=False)
trainProcesado = pd.concat([trainProcesado, dummies], axis=1)

trainProcesado.drop('antiguedad_agrupada', 1, inplace = True)
trainProcesado.drop('metroscubiertos_agrupada', 1, inplace = True)
trainProcesado.drop('metrostotales_agrupada', 1, inplace = True)

trainProcesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Columns: 129 entries, id to metrostotales_agrupada_90
dtypes: category(2), float64(6), int64(16), uint8(105)
memory usage: 64.8 MB


In [11]:
#train_gruouped = trainProcesado.groupby(['idzona','tipodepropiedad','habitaciones','anio'])['precio'].agg({'mean','median'}).reset_index().rename(columns={"mean": "mean_propiedad", "median": "median_propiedad"})
#train_gruouped_2 = trainProcesado.groupby(['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'])['precio'].agg({'mean','median'}).reset_index().rename(columns={"mean": "mean_agrupado", "median": "median_agrupado"})

train_gruouped_3 = trainProcesado.groupby(['idzona','tipodepropiedad','habitaciones','anio'])['precio_2016'].agg({'mean','median'}).reset_index().rename(columns={"mean": "mean_2016", "median": "median_2016"})
#train_gruouped_4 = trainProcesado.groupby(['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'])['precio_2016'].agg({'mean','median'}).reset_index().rename(columns={"mean": "mean_2016_agrupado", "median": "median_2016_agrupado"})

#trainProcesado = pd.merge(trainProcesado, train_gruouped, how='left', left_on=['idzona','tipodepropiedad','habitaciones','anio'], right_on = ['idzona','tipodepropiedad','habitaciones','anio'])
#trainProcesado = pd.merge(trainProcesado, train_gruouped_2, how='left', left_on=['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'], right_on = ['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'])

trainProcesado = pd.merge(trainProcesado, train_gruouped_3, how='left', left_on=['idzona','tipodepropiedad','habitaciones','anio'], right_on = ['idzona','tipodepropiedad','habitaciones','anio'])
#trainProcesado = pd.merge(trainProcesado, train_gruouped_4, how='left', left_on=['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'], right_on = ['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'])


#trainProcesado['mean_propiedad'].fillna(trainProcesado['mean_propiedad'].mean(), inplace = True)
#trainProcesado['median_propiedad'].fillna(trainProcesado['median_propiedad'].mean(), inplace = True)

#trainProcesado['median_agrupado'].fillna(trainProcesado['median_agrupado'].mean(), inplace = True)
#trainProcesado['mean_agrupado'].fillna(trainProcesado['mean_agrupado'].mean(), inplace = True)

trainProcesado['mean_2016'].fillna(trainProcesado['mean_2016'].mean(), inplace = True)
trainProcesado['median_2016'].fillna(trainProcesado['median_2016'].mean(), inplace = True)

trainProcesado['mean_2016'] = trainProcesado['mean_2016'].astype(int)
trainProcesado['median_2016'] = trainProcesado['median_2016'].astype(int)

#trainProcesado['mean_2016_agrupado'].fillna(trainProcesmedian_2016do['mean_2016_agrupado'].mean(), inplace = True)
#trainProcesado['median_2016_agrupado'].fillna(trainProcesado['median_2016_agrupado'].mean(), inplace = True)

In [12]:

#AGRUPAMIENTO POR mean_2016 (BANDAS)
trainProcesado['mean_2016_agrupado'] = 'mean_2016_agrupado_0'

trainProcesado.loc[trainProcesado.mean_2016 <= 500000, 'mean_2016_agrupado'] = 'mean_2016_agrupado_0'
trainProcesado.loc[trainProcesado.mean_2016 > 11000000, 'mean_2016_agrupado'] = 'mean_2016_agrupado_22'
trainProcesado.loc[((trainProcesado.mean_2016 > 500000) & (trainProcesado.mean_2016 <= 1000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_1'
trainProcesado.loc[((trainProcesado.mean_2016 > 1000000) & (trainProcesado.mean_2016 <= 1500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_2'
trainProcesado.loc[((trainProcesado.mean_2016 > 1500000) & (trainProcesado.mean_2016 <= 2000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_3'
trainProcesado.loc[((trainProcesado.mean_2016 > 2000000) & (trainProcesado.mean_2016 <= 2500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_4'
trainProcesado.loc[((trainProcesado.mean_2016 > 2500000) & (trainProcesado.mean_2016 <= 3000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_5'
trainProcesado.loc[((trainProcesado.mean_2016 > 3000000) & (trainProcesado.mean_2016 <= 3500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_6'
trainProcesado.loc[((trainProcesado.mean_2016 > 3500000) & (trainProcesado.mean_2016 <= 4000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_7'
trainProcesado.loc[((trainProcesado.mean_2016 > 4000000) & (trainProcesado.mean_2016 <= 4500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_8'
trainProcesado.loc[((trainProcesado.mean_2016 > 4500000) & (trainProcesado.mean_2016 <= 5000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_9'
trainProcesado.loc[((trainProcesado.mean_2016 > 5000000) & (trainProcesado.mean_2016 <= 5500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_10'
trainProcesado.loc[((trainProcesado.mean_2016 > 5500000) & (trainProcesado.mean_2016 <= 6000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_11'
trainProcesado.loc[((trainProcesado.mean_2016 > 6000000) & (trainProcesado.mean_2016 <= 6500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_12'
trainProcesado.loc[((trainProcesado.mean_2016 > 6500000) & (trainProcesado.mean_2016 <= 7000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_13'
trainProcesado.loc[((trainProcesado.mean_2016 > 7000000) & (trainProcesado.mean_2016 <= 7500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_14'
trainProcesado.loc[((trainProcesado.mean_2016 > 7500000) & (trainProcesado.mean_2016 <= 8000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_15'
trainProcesado.loc[((trainProcesado.mean_2016 > 8000000) & (trainProcesado.mean_2016 <= 8500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_16'
trainProcesado.loc[((trainProcesado.mean_2016 > 8500000) & (trainProcesado.mean_2016 <= 9000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_17'
trainProcesado.loc[((trainProcesado.mean_2016 > 9000000) & (trainProcesado.mean_2016 <= 9500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_18'
trainProcesado.loc[((trainProcesado.mean_2016 > 9500000) & (trainProcesado.mean_2016 <= 10000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_19'
trainProcesado.loc[((trainProcesado.mean_2016 > 10000000) & (trainProcesado.mean_2016 <= 10500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_20'
trainProcesado.loc[((trainProcesado.mean_2016 > 10500000) & (trainProcesado.mean_2016 <= 11000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_21'

# trainProcesado.drop('antiguedad', 1, inplace = True)

In [13]:
trainProcesado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Columns: 132 entries, id to mean_2016_agrupado
dtypes: category(2), float64(6), int64(18), object(1), uint8(105)
memory usage: 72.1+ MB


In [14]:
#Procesamiento de columna titulo
#Parseamos las palabras seleccionadas en el DF (ver notebook de exploracion)

#Nuevo dataframe
titleFeatures = pd.DataFrame()
titleFeatures['id'] = train['id']
#Uppercase
titleFeatures['titulo'] = train['titulo'].str.upper()
#Limpiamos las columnas
titleFeatures['titulo'] = titleFeatures['titulo'].str.normalize('NFKD')\
                                                .str.encode('ascii', errors='ignore')\
                                                .str.decode('utf-8')

#Armamos las columnas para cada palabra seleccionada
#Valor de inicializacion

#Propiedad nueva
titleFeatures['NUEVA'] = 0
#Calificativo (no aporta mucho)
titleFeatures['EXCELENTE'] = 0
#Tipo de aviso
titleFeatures['VENTA'] = 0
titleFeatures['PREVENTA'] = 0
titleFeatures['RENTA'] = 0
titleFeatures['CREDITO'] = 0
titleFeatures['REMATE'] = 0
titleFeatures['OPORTUNIDAD'] = 0
#Tipo de propiedad -extra
titleFeatures['RESIDENCIA'] = 0
titleFeatures['EDIFICIO'] = 0
titleFeatures['TORRE'] = 0
titleFeatures['PISO'] = 0
titleFeatures['LOFT'] = 0
titleFeatures['FRACCIONAMIENTO'] = 0
titleFeatures['DUPLEX'] = 0
titleFeatures['COUNTRY'] = 0
titleFeatures['QUINTA'] = 0
titleFeatures['OFICINA'] = 0
titleFeatures['PENTHOUSE'] = 0
#Tipo de propiedad -extra 2
titleFeatures['REMODELADO'] = 0
titleFeatures['MODERNO'] = 0
titleFeatures['MINIMALISTA'] = 0
titleFeatures['EQUIPADA'] = 0
#Jardin
titleFeatures['JARDIN'] = 0
#Ubicacion
titleFeatures['FRENTE'] = 0
titleFeatures['ESQUINA'] = 0
titleFeatures['CENTRO'] = 0
titleFeatures['UBICACION'] = 0
#Vistas o panoramicas
titleFeatures['VISTA'] = 0
#Caracteristicas extra
titleFeatures['LUJO'] = 0
titleFeatures['BODEGA'] = 0
titleFeatures['CAMPESTRE'] = 0
titleFeatures['RANCHO'] = 0
titleFeatures['PLAYA'] = 0
titleFeatures['GOLF'] = 0
#Seguridad
titleFeatures['SEGURIDAD'] = 0

#Buscamos y agregamos 1 donde aparece la palabra
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVA', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVO', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVAS', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENE', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENAR', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVOS', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENA', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXELENTE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('BONITO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTES', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('INMEJORABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('AGRADABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('IMPECABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSAS', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESPECTACULAR', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('INCREIBLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('SUPER', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VENTA', na=False), 'VENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PREVENTA', na=False), 'PREVENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRE-VENTA', na=False), 'PREVENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RENTA', na=False), 'RENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CREDITO', na=False), 'CREDITO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HIPOTECARIO', na=False), 'CREDITO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATE', na=False), 'REMATE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATO', na=False), 'REMATE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('OPORTUNIDAD', na=False), 'OPORTUNIDAD'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIA', na=False), 'RESIDENCIA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIAL', na=False), 'RESIDENCIA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EDIFICIO', na=False), 'EDIFICIO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('TORRE', na=False), 'TORRE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PISO', na=False), 'PISO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LOFT', na=False), 'LOFT'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACCIONAMIENTO', na=False), 'FRACCIONAMIENTO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACC', na=False), 'FRACCIONAMIENTO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('DUPLEX', na=False), 'DUPLEX'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('COUNTRY', na=False), 'COUNTRY'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('QUINTA', na=False), 'QUINTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('OFICINA', na=False), 'OFICINA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PENTHOUSE', na=False), 'PENTHOUSE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADO', na=False), 'REMODELADO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADA', na=False), 'REMODELADO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNA', na=False), 'MODERNO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNO', na=False), 'MODERNO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MINIMALISTA', na=False), 'MINIMALISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EQUIPADA', na=False), 'EQUIPADA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDINES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDIN', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VERDES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ARBOLEDAS', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PARQUES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PATIO', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRENTE', na=False), 'FRENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESQUINA', na=False), 'ESQUINA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRO', na=False), 'CENTRO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRAL', na=False), 'CENTRO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICACION', na=False), 'UBICACION'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICADISIMO', na=False), 'UBICACION'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PANORAMICA', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTA', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTAS', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LUJO', na=False), 'LUJO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('BODEGA', na=False), 'BODEGA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CAMPESTRE', na=False), 'CAMPESTRE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RANCHO', na=False), 'RANCHO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PLAYA', na=False), 'PLAYA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('GOLF', na=False), 'GOLF'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('SEGURIDAD', na=False), 'SEGURIDAD'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VIGILANCIA', na=False), 'SEGURIDAD'] = 1

#Alternativamente, llenamos una sola columna como categorica, para luego encodear con LE
titleFeatures['titulo_features'] = 'NINGUNA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVA', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVO', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVAS', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENE', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENAR', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVOS', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENA', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXELENTE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('BONITO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTES', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('INMEJORABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('AGRADABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('IMPECABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSAS', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESPECTACULAR', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('INCREIBLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('SUPER', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VENTA', na=False), 'titulo_features'] = 'VENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PREVENTA', na=False), 'titulo_features'] = 'PREVENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRE-VENTA', na=False), 'titulo_features'] = 'PREVENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RENTA', na=False), 'titulo_features'] = 'RENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CREDITO', na=False), 'titulo_features'] = 'CREDITO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HIPOTECARIO', na=False), 'titulo_features'] = 'CREDITO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATE', na=False), 'titulo_features'] = 'REMATE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATO', na=False), 'titulo_features'] = 'REMATE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('OPORTUNIDAD', na=False), 'titulo_features'] = 'OPORTUNIDAD'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIA', na=False), 'titulo_features'] = 'RESIDENCIA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIAL', na=False), 'titulo_features'] = 'RESIDENCIA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EDIFICIO', na=False), 'titulo_features'] = 'EDIFICIO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('TORRE', na=False), 'titulo_features'] = 'TORRE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PISO', na=False), 'titulo_features'] = 'PISO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LOFT', na=False), 'titulo_features'] = 'LOFT'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACCIONAMIENTO', na=False), 'titulo_features'] = 'FRACCIONAMIENTO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACC', na=False), 'titulo_features'] = 'FRACCIONAMIENTO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('DUPLEX', na=False), 'titulo_features'] = 'DUPLEX'
titleFeatures.loc[titleFeatures['titulo'].str.contains('COUNTRY', na=False), 'titulo_features'] = 'COUNTRY'
titleFeatures.loc[titleFeatures['titulo'].str.contains('QUINTA', na=False), 'titulo_features'] = 'QUINTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('OFICINA', na=False), 'titulo_features'] = 'OFICINA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PENTHOUSE', na=False), 'titulo_features'] = 'PENTHOUSE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADO', na=False), 'titulo_features'] = 'REMODELADO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADA', na=False), 'titulo_features'] = 'REMODELADO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNA', na=False), 'titulo_features'] = 'MODERNO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNO', na=False), 'titulo_features'] = 'MODERNO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MINIMALISTA', na=False), 'titulo_features'] = 'MINIMALISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EQUIPADA', na=False), 'titulo_features'] = 'EQUIPADA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDINES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDIN', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VERDES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ARBOLEDAS', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PARQUES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PATIO', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRENTE', na=False), 'titulo_features'] = 'FRENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESQUINA', na=False), 'titulo_features'] = 'ESQUINA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRO', na=False), 'titulo_features'] = 'CENTRO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRAL', na=False), 'titulo_features'] = 'CENTRO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICACION', na=False), 'titulo_features'] = 'UBICACION'
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICADISIMO', na=False), 'titulo_features'] = 'UBICACION'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PANORAMICA', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTA', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTAS', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LUJO', na=False), 'titulo_features'] = 'LUJO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('BODEGA', na=False), 'titulo_features'] = 'BODEGA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CAMPESTRE', na=False), 'titulo_features'] = 'CAMPESTRE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RANCHO', na=False), 'titulo_features'] = 'RANCHO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PLAYA', na=False), 'titulo_features'] = 'PLAYA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('GOLF', na=False), 'titulo_features'] = 'GOLF'
titleFeatures.loc[titleFeatures['titulo'].str.contains('SEGURIDAD', na=False), 'titulo_features'] = 'SEGURIDAD'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VIGILANCIA', na=False), 'titulo_features'] = 'SEGURIDAD'
#La tipamos
titleFeatures['titulo_features'] = titleFeatures['titulo_features'].astype('category')

#La guardamos con label encoding
cat_features_titulo = ['titulo_features']
encoder_titulo = LabelEncoder()
encoded_titulo = titleFeatures[cat_features_titulo].apply(encoder_titulo.fit_transform)
encoded_titulo['id'] = train['id']
encoded_titulo = encoded_titulo.rename(columns={'titulo_features': 'titulo_features_enc'})

#Agregamos la columna al los DF
titleFeatures = pd.merge(titleFeatures,encoded_titulo, on='id', how='inner')
trainProcesado = pd.merge(trainProcesado,titleFeatures, on='id', how='inner')
trainProcesado = trainProcesado.drop('titulo',1)
trainProcesado = trainProcesado.drop('titulo_features',1)
trainProcesado = trainProcesado.drop('titulo_features_enc',1)

In [15]:
#Tratamiento analogo para la columna descripcion

#Procesamiento de columna descripcion
#Parseamos las palabras seleccionadas en el DF (ver notebook de exploracion)

#Nuevo dataframe
descFeatures = pd.DataFrame()
descFeatures['id'] = train['id']
#Uppercase
descFeatures['descripcion'] = train['descripcion'].str.upper()
#Limpiamos las columnas
descFeatures['descripcion'] = descFeatures['descripcion'].str.normalize('NFKD')\
                                                .str.encode('ascii', errors='ignore')\
                                                .str.decode('utf-8')

#Armamos las columnas para cada palabra seleccionada
#Valor de inicializacion

#Ya usadas en titulo
#Propiedad nueva
descFeatures['NUEVA'] = 0
#Calificativo (no aporta mucho)
descFeatures['EXCELENTE'] = 0
#Tipo de aviso
descFeatures['VENTA'] = 0
descFeatures['PREVENTA'] = 0
descFeatures['RENTA'] = 0
descFeatures['CREDITO'] = 0
descFeatures['REMATE'] = 0
descFeatures['OPORTUNIDAD'] = 0
#Tipo de propiedad -extra
descFeatures['RESIDENCIA'] = 0
descFeatures['EDIFICIO'] = 0
descFeatures['TORRE'] = 0
descFeatures['PISO'] = 0
descFeatures['LOFT'] = 0
descFeatures['FRACCIONAMIENTO'] = 0
descFeatures['DUPLEX'] = 0
descFeatures['COUNTRY'] = 0
descFeatures['QUINTA'] = 0
descFeatures['OFICINA'] = 0
descFeatures['PENTHOUSE'] = 0
#Tipo de propiedad -extra 2
descFeatures['REMODELADO'] = 0
descFeatures['MODERNO'] = 0
descFeatures['MINIMALISTA'] = 0
descFeatures['EQUIPADA'] = 0
#Jardin
descFeatures['JARDIN'] = 0
#Ubicacion
descFeatures['FRENTE'] = 0
descFeatures['ESQUINA'] = 0
descFeatures['CENTRO'] = 0
descFeatures['UBICACION'] = 0
#Vistas o panoramicas
descFeatures['VISTA'] = 0
#Caracteristicas extra
descFeatures['LUJO'] = 0
descFeatures['BODEGA'] = 0
descFeatures['CAMPESTRE'] = 0
descFeatures['RANCHO'] = 0
descFeatures['PLAYA'] = 0
descFeatures['GOLF'] = 0
#Seguridad
descFeatures['SEGURIDAD'] = 0

#Nuevas de descripcion
#Iluminacion
descFeatures['ILUMINADA'] = 0
#Ubicacion
descFeatures['TRANQUILO'] = 0
descFeatures['UNIVERSIDAD'] = 0
descFeatures['AVENIDA'] = 0
#Caracteristicas extra
descFeatures['AMENIDADES'] = 0
descFeatures['BARRA'] = 0
descFeatures['BALCON'] = 0
descFeatures['RECIBIDOR'] = 0
descFeatures['AMENIDADES'] = 0
descFeatures['DESPENSA'] = 0
descFeatures['LAVADERO'] = 0
descFeatures['REJA'] = 0
descFeatures['TERRAZA'] = 0
descFeatures['CHIMENEA'] = 0
descFeatures['CISTERNA'] = 0

#Buscamos y agregamos 1 donde aparece la palabra
descFeatures.loc[descFeatures['descripcion'].str.contains('NUEVO', na=False), 'NUEVA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTRENAR', na=False), 'NUEVA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCELENTE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSO', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AGRADABLE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSO', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESPECTACULAR', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSAS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSOS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITOS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INMEJORABLE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMATES', na=False), 'REMATE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HIPOTECARIO', na=False), 'CREDITO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSION', na=False), 'OPORTUNIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSIONISTAS', na=False), 'OPORTUNIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('RESIDENCIAL', na=False), 'RESIDENCIA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FRACCIONAMIENTO', na=False), 'FRACCIONAMIENTO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTANCIA', na=False), 'RANCHO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PISO', na=False), 'PISO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TORRE', na=False), 'TORRE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LUZ', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADA', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADO', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINACION', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PATIO', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FONDO', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JARDIN', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDE', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('GARDEN', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUE', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('NATURAL', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BOSQUES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ARBOLES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VIGILANCIA', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURIDAD', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURO', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTAS', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PANORAMICA', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTA', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICACION', na=False), 'UBICACION'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICADO', na=False), 'UBICACION'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILA', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILO', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILIDAD', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UNIVERSIDAD', na=False), 'UNIVERSIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AVENIDA', na=False), 'AVENIDA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESQUINA', na=False), 'ESQUINA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FACHADA', na=False), 'FRENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FRENTE', na=False), 'FRENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADO', na=False), 'REMODELADO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADA', na=False), 'REMODELADO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('MODERNA', na=False), 'MODERNO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCLUSIVA', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LUJO', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CALIDAD', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('MARMOL', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PORCELANATO', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JACUZZI', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIDOR', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('GOLF', na=False), 'GOLF'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BODEGA', na=False), 'BODEGA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PLAYA', na=False), 'PLAYA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AMENIDADES ', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FIESTAS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EVENTOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVANDERIA', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JUEGOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIO', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ASADOR', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CANCHAS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TENIS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BARRA', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADORA', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADOR', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BALCON', na=False), 'BALCON'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('RECIBIDOR', na=False), 'RECIBIDOR'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIBULO', na=False), 'RECIBIDOR'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CHIMENEA', na=False), 'CHIMENEA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESPENSA', na=False), 'DESPENSA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVADERO', na=False), 'LAVADERO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REJA', na=False), 'REJA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BOILER', na=False), 'CISTERNA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CISTERNA', na=False), 'CISTERNA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AZOTEA', na=False), 'TERRAZA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TERRAZA', na=False), 'TERRAZA'] = 1

#Alternativamente, llenamos una sola columna como categorica, para luego encodear con LE
descFeatures['descripcion_features'] = 'NINGUNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('NUEVO', na=False), 'descripcion_features'] = 'NUEVA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTRENAR', na=False), 'descripcion_features'] = 'NUEVA'
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCELENTE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSO', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('AGRADABLE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSO', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESPECTACULAR', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSAS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSOS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITOS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('INMEJORABLE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMATES', na=False), 'descripcion_features'] = 'REMATE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HIPOTECARIO', na=False), 'descripcion_features'] = 'CREDITO'
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSION', na=False), 'descripcion_features'] = 'OPORTUNIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSIONISTAS', na=False), 'descripcion_features'] = 'OPORTUNIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('RESIDENCIAL', na=False), 'descripcion_features'] = 'RESIDENCIA'
descFeatures.loc[descFeatures['descripcion'].str.contains('FRACCIONAMIENTO', na=False), 'descripcion_features'] = 'FRACCIONAMIENTO'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTANCIA', na=False), 'descripcion_features'] = 'RANCHO'
descFeatures.loc[descFeatures['descripcion'].str.contains('PISO', na=False), 'descripcion_features'] = 'PISO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TORRE', na=False), 'descripcion_features'] = 'TORRE'
descFeatures.loc[descFeatures['descripcion'].str.contains('LUZ', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADA', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADO', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINACION', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PATIO', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('FONDO', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('JARDIN', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDE', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('GARDEN', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUE', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('NATURAL', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('BOSQUES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('ARBOLES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VIGILANCIA', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURIDAD', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURO', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTAS', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PANORAMICA', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTA', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICACION', na=False), 'descripcion_features'] = 'UBICACION'
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICADO', na=False), 'descripcion_features'] = 'UBICACION'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILA', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILO', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILIDAD', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('UNIVERSIDAD', na=False), 'descripcion_features'] = 'UNIVERSIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('AVENIDA', na=False), 'descripcion_features'] = 'AVENIDA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESQUINA', na=False), 'descripcion_features'] = 'ESQUINA'
descFeatures.loc[descFeatures['descripcion'].str.contains('FACHADA', na=False), 'descripcion_features'] = 'FRENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('FRENTE', na=False), 'descripcion_features'] = 'FRENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADO', na=False), 'descripcion_features'] = 'REMODELADO'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADA', na=False), 'descripcion_features'] = 'REMODELADO'
descFeatures.loc[descFeatures['descripcion'].str.contains('MODERNA', na=False), 'descripcion_features'] = 'MODERNO'
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCLUSIVA', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('LUJO', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('CALIDAD', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('MARMOL', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('PORCELANATO', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('JACUZZI', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIDOR', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('GOLF', na=False), 'descripcion_features'] = 'GOLF'
descFeatures.loc[descFeatures['descripcion'].str.contains('BODEGA', na=False), 'descripcion_features'] = 'BODEGA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PLAYA', na=False), 'descripcion_features'] = 'PLAYA'
descFeatures.loc[descFeatures['descripcion'].str.contains('AMENIDADES ', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('FIESTAS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('EVENTOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVANDERIA', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('JUEGOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIO', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('ASADOR', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('CANCHAS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('TENIS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('BARRA', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADORA', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADOR', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('BALCON', na=False), 'descripcion_features'] = 'BALCON'
descFeatures.loc[descFeatures['descripcion'].str.contains('RECIBIDOR', na=False), 'descripcion_features'] = 'RECIBIDOR'
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIBULO', na=False), 'descripcion_features'] = 'RECIBIDOR'
descFeatures.loc[descFeatures['descripcion'].str.contains('CHIMENEA', na=False), 'descripcion_features'] = 'CHIMENEA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESPENSA', na=False), 'descripcion_features'] = 'DESPENSA'
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVADERO', na=False), 'descripcion_features'] = 'LAVADERO'
descFeatures.loc[descFeatures['descripcion'].str.contains('REJA', na=False), 'descripcion_features'] = 'REJA'
descFeatures.loc[descFeatures['descripcion'].str.contains('BOILER', na=False), 'descripcion_features'] = 'CISTERNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('CISTERNA', na=False), 'descripcion_features'] = 'CISTERNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('AZOTEA', na=False), 'descripcion_features'] = 'TERRAZA'
descFeatures.loc[descFeatures['descripcion'].str.contains('TERRAZA', na=False), 'descripcion_features'] = 'TERRAZA'

#La tipamos
descFeatures['descripcion_features'] = descFeatures['descripcion_features'].astype('category')

#La guardamos con label encoding
cat_features_desc = ['descripcion_features']
encoder_desc = LabelEncoder()
encoded_desc = descFeatures[cat_features_desc].apply(encoder_desc.fit_transform)
encoded_desc['id'] = train['id']
encoded_desc = encoded_desc.rename(columns={'descripcion_features': 'descripcion_features_enc'})

#Agregamos la columna al los DF
descFeatures = pd.merge(descFeatures,encoded_desc, on='id', how='inner')
trainProcesado = pd.merge(trainProcesado,descFeatures, on='id', how='inner')
trainProcesado = trainProcesado.drop('descripcion',1)
trainProcesado = trainProcesado.drop('descripcion_features',1)
trainProcesado = trainProcesado.drop('descripcion_features_enc',1)

In [16]:
#OR entre las columnas repetidas en ambos DF
#Propiedad nueva
trainProcesado['NUEVA'] = trainProcesado['NUEVA_x'] | trainProcesado['NUEVA_y']
#Calificativo (no aporta mucho)
trainProcesado['EXCELENTE'] = trainProcesado['EXCELENTE_x'] | trainProcesado['EXCELENTE_y']
#Tipo de aviso
trainProcesado['VENTA'] = trainProcesado['VENTA_x'] | trainProcesado['VENTA_y'] 
trainProcesado['PREVENTA'] = trainProcesado['PREVENTA_x'] | trainProcesado['PREVENTA_y']
trainProcesado['RENTA'] = trainProcesado['RENTA_x'] | trainProcesado['RENTA_y'] 
trainProcesado['CREDITO'] = trainProcesado['CREDITO_x'] | trainProcesado['CREDITO_y']
trainProcesado['REMATE'] = trainProcesado['REMATE_x'] | trainProcesado['REMATE_y']
trainProcesado['OPORTUNIDAD'] = trainProcesado['OPORTUNIDAD_x'] | trainProcesado['OPORTUNIDAD_y']
#Tipo de propiedad -extra
trainProcesado['RESIDENCIA'] = trainProcesado['RESIDENCIA_x'] | trainProcesado['RESIDENCIA_y']
trainProcesado['EDIFICIO'] = trainProcesado['EDIFICIO_x'] | trainProcesado['EDIFICIO_y'] 
trainProcesado['TORRE'] = trainProcesado['TORRE_x'] | trainProcesado['TORRE_y']
trainProcesado['PISO'] = trainProcesado['PISO_x'] | trainProcesado['PISO_y']
trainProcesado['LOFT'] = trainProcesado['LOFT_x'] | trainProcesado['LOFT_y']
trainProcesado['FRACCIONAMIENTO'] = trainProcesado['FRACCIONAMIENTO_x'] | trainProcesado['FRACCIONAMIENTO_y']
trainProcesado['DUPLEX'] = trainProcesado['DUPLEX_x'] | trainProcesado['DUPLEX_y'] 
trainProcesado['COUNTRY'] = trainProcesado['COUNTRY_x'] | trainProcesado['COUNTRY_y']
trainProcesado['QUINTA'] = trainProcesado['QUINTA_x'] | trainProcesado['QUINTA_y']
trainProcesado['OFICINA'] = trainProcesado['OFICINA_x'] | trainProcesado['OFICINA_y']
trainProcesado['PENTHOUSE'] = trainProcesado['PENTHOUSE_x'] | trainProcesado['PENTHOUSE_y']
#Tipo de propiedad -extra 2
trainProcesado['REMODELADO'] = trainProcesado['REMODELADO_x'] | trainProcesado['REMODELADO_y']
trainProcesado['MODERNO'] = trainProcesado['MODERNO_x'] | trainProcesado['MODERNO_y']
trainProcesado['MINIMALISTA'] = trainProcesado['MINIMALISTA_x'] | trainProcesado['MINIMALISTA_y']
trainProcesado['EQUIPADA'] = trainProcesado['EQUIPADA_x'] | trainProcesado['EQUIPADA_y']
#Jardin
trainProcesado['JARDIN'] = trainProcesado['JARDIN_x'] | trainProcesado['JARDIN_y'] 
#Ubicacion
trainProcesado['FRENTE'] = trainProcesado['FRENTE_x'] | trainProcesado['FRENTE_y'] 
trainProcesado['ESQUINA'] = trainProcesado['ESQUINA_x'] | trainProcesado['ESQUINA_y'] 
trainProcesado['CENTRO'] = trainProcesado['CENTRO_x'] | trainProcesado['CENTRO_y']
trainProcesado['UBICACION'] = trainProcesado['UBICACION_x'] | trainProcesado['UBICACION_y']
#Vistas o panoramicas
trainProcesado['VISTA'] = trainProcesado['VISTA_x'] | trainProcesado['VISTA_y']
#Caracteristicas extra
trainProcesado['LUJO'] = trainProcesado['LUJO_x'] |  trainProcesado['LUJO_y']
trainProcesado['BODEGA'] = trainProcesado['BODEGA_x'] | trainProcesado['BODEGA_y']
trainProcesado['CAMPESTRE'] = trainProcesado['CAMPESTRE_x'] | trainProcesado['CAMPESTRE_x']
trainProcesado['RANCHO'] = trainProcesado['RANCHO_x'] | trainProcesado['RANCHO_y'] 
trainProcesado['PLAYA'] = trainProcesado['PLAYA_x'] | trainProcesado['PLAYA_y']
trainProcesado['GOLF'] = trainProcesado['GOLF_x'] | trainProcesado['GOLF_y']
#Seguridad
trainProcesado['SEGURIDAD'] = trainProcesado['SEGURIDAD_x'] | trainProcesado['SEGURIDAD_y'] 

#Borramos las columnas que quedaron del merge
trainProcesado.drop(list(trainProcesado.filter(regex = '_x')), axis = 1, inplace = True)
trainProcesado.drop(list(trainProcesado.filter(regex = '_y')), axis = 1, inplace = True)

In [17]:
trainProcesado.drop('idzona', axis = 1, inplace = True)
trainProcesado.drop('anio', axis = 1, inplace = True)

trainProcesado.drop('precio_2012', axis = 1, inplace = True)
trainProcesado.drop('precio_2013', axis = 1, inplace = True)
trainProcesado.drop('precio_2014', axis = 1, inplace = True)
trainProcesado.drop('precio_2015', axis = 1, inplace = True)
trainProcesado.drop('precio_2016', axis = 1, inplace = True)
trainProcesado.drop('GARAGE', axis = 1, inplace = True)
trainProcesado.drop('HOSPEDAJE', axis = 1, inplace = True)
trainProcesado.drop('provincia', axis = 1, inplace = True)
trainProcesado.drop('tipodepropiedad', axis = 1, inplace = True)

dummies = pd.get_dummies(trainProcesado['mean_2016_agrupado'], drop_first=False)
trainProcesado = pd.concat([trainProcesado, dummies], axis=1)
trainProcesado.drop('mean_2016_agrupado', axis = 1, inplace = True)

In [18]:
#Exportamos el DF a CSV, para usar como entrada de los distintos algoritmos probados
trainProcesado.to_csv('DATA/train_procesado.csv', index=False)

# TEST

In [19]:
#Lectura de archivos CSV
test = pd.read_csv('../../DATA_TP2/test.csv')

In [20]:
#Nuevo dataframe
testProcesado = pd.DataFrame()

#Copiamos la columna con el id
testProcesado['id'] = test['id']
testProcesado['idzona'] = test['idzona']

#Manejo de NaN, columnas numericas
test['garages'].fillna(0, inplace=True)
test['banos'].fillna(1, inplace=True)

promedio_antiguedad_x_ciudad = test.groupby(['ciudad','idzona'])['antiguedad'].transform('mean')
test['antiguedad'].fillna(promedio_antiguedad_x_ciudad, inplace=True)
test['antiguedad'].fillna(test['antiguedad'].mean(), inplace=True)

promedio_habitaciones_x_ciudad = test.groupby(['ciudad','idzona'])['habitaciones'].transform('mean')
test['habitaciones'].fillna(promedio_habitaciones_x_ciudad, inplace=True)
test['habitaciones'].fillna(test['habitaciones'].mean(), inplace=True)

promedio_metroscubierto_x_ciudad = test.groupby(['ciudad','idzona'])['metroscubiertos'].transform('mean')
test['metroscubiertos'].fillna(promedio_metroscubierto_x_ciudad, inplace=True)
test['metroscubiertos'].fillna(test['metroscubiertos'].mean(), inplace=True)

promedio_metrostotales_x_ciudad = test.groupby(['ciudad','idzona'])['metrostotales'].transform('mean')
test['metrostotales'].fillna(promedio_metrostotales_x_ciudad, inplace=True)
test['metrostotales'].fillna(test['metrostotales'].mean(), inplace=True)

#Manejo de NaN, columnas catedoricas
test['ciudad'] = test['ciudad'].fillna('Desconocida')
test['provincia'] = test['provincia'].fillna('Desconocida')
test['tipodepropiedad'] = test['tipodepropiedad'].fillna('Desconocido')

test['ciudad'] = test['ciudad'].str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
test['provincia'] = test['provincia'].str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
test['tipodepropiedad'] = test['tipodepropiedad'].str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

test['ciudad'] = test['ciudad'].astype('category')
test['provincia'] = test['provincia'].astype('category')
test['tipodepropiedad'] = test['tipodepropiedad'].astype('category')

test['ciudad'] = test['ciudad'].astype('category')
testProcesado['provincia'] = test['provincia'].astype('category')
testProcesado['tipodepropiedad'] = test['tipodepropiedad'].astype('category')

#Asignacion de tipos de datos
testProcesado['antiguedad'] = test['antiguedad'].astype('int64')
testProcesado['habitaciones'] = test['habitaciones'].astype('int64')
testProcesado['garages'] = test['garages'].astype('int64')
testProcesado['banos'] = test['banos'].astype('int64')
testProcesado['metroscubiertos'] = test['metroscubiertos'].astype('int64')
testProcesado['metrostotales'] = test['metrostotales'].astype('int64')

testProcesado['gimnasio'] = test['gimnasio'].astype('int64')
testProcesado['usosmultiples'] = test['usosmultiples'].astype('int64')
testProcesado['piscina'] = test['piscina'].astype('int64')
testProcesado['escuelascercanas'] = test['escuelascercanas'].astype('int64')
testProcesado['escuelascercanas'] = test['escuelascercanas'].astype('int64')
testProcesado['centroscomercialescercanos'] = test['centroscomercialescercanos'].astype('int64')

#Solo nos quedamos con el anio de la fecha
testProcesado['anio'] = pd.to_datetime(test['fecha'], infer_datetime_format=True).dt.year

testProcesado['2012'] = 0
testProcesado['2013'] = 0
testProcesado['2014'] = 0
testProcesado['2015'] = 0
testProcesado['2016'] = 0

testProcesado.loc[test['fecha'].str.contains('2012', na=False), '2012'] = 1
testProcesado.loc[test['fecha'].str.contains('2013', na=False), '2013'] = 1
testProcesado.loc[test['fecha'].str.contains('2014', na=False), '2014'] = 1
testProcesado.loc[test['fecha'].str.contains('2015', na=False), '2015'] = 1
testProcesado.loc[test['fecha'].str.contains('2016', na=False), '2016'] = 1

In [21]:
dummies = pd.get_dummies(testProcesado['provincia'], drop_first=False)
testProcesado = pd.concat([testProcesado, dummies], axis=1)

dummies = pd.get_dummies(testProcesado['tipodepropiedad'], drop_first=False)
testProcesado = pd.concat([testProcesado, dummies], axis=1)

In [22]:
#AGRUPAMIENTO POR ANTIGUEDAD (BANDAS)
testProcesado['antiguedad_agrupada'] = 'antiguedad_agrupada_0'

testProcesado.loc[testProcesado.antiguedad <= 5, 'antiguedad_agrupada'] = 'antiguedad_agrupada_0'
testProcesado.loc[testProcesado.antiguedad > 95, 'antiguedad_agrupada'] = 'antiguedad_agrupada_100'
testProcesado.loc[((testProcesado.antiguedad > 5) & (testProcesado.antiguedad <= 15)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_10'
testProcesado.loc[((testProcesado.antiguedad > 15) & (testProcesado.antiguedad <= 25)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_20'
testProcesado.loc[((testProcesado.antiguedad > 25) & (testProcesado.antiguedad <= 35)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_30'
testProcesado.loc[((testProcesado.antiguedad > 35) & (testProcesado.antiguedad <= 45)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_40'
testProcesado.loc[((testProcesado.antiguedad > 45) & (testProcesado.antiguedad <= 55)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_50'
testProcesado.loc[((testProcesado.antiguedad > 55) & (testProcesado.antiguedad <= 65)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_60'
testProcesado.loc[((testProcesado.antiguedad > 65) & (testProcesado.antiguedad <= 75)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_70'
testProcesado.loc[((testProcesado.antiguedad > 75) & (testProcesado.antiguedad <= 85)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_80'
testProcesado.loc[((testProcesado.antiguedad > 85) & (testProcesado.antiguedad <= 95)), 'antiguedad_agrupada'] = 'antiguedad_agrupada_90'

testProcesado.drop('antiguedad', 1, inplace = True)

# AGRUPAMIENTO POR METROS CUBIENTOS (BANDAS)
testProcesado['metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_0'

testProcesado.loc[testProcesado.metroscubiertos <= 5, 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_0'
testProcesado.loc[testProcesado.metroscubiertos > 805, 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_1000'
testProcesado.loc[((testProcesado.metroscubiertos > 5) & (testProcesado.metroscubiertos <= 15)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_10'
testProcesado.loc[((testProcesado.metroscubiertos > 15) & (testProcesado.metroscubiertos <= 25)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_20'
testProcesado.loc[((testProcesado.metroscubiertos > 25) & (testProcesado.metroscubiertos <= 35)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_30'
testProcesado.loc[((testProcesado.metroscubiertos > 35) & (testProcesado.metroscubiertos <= 45)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_40'
testProcesado.loc[((testProcesado.metroscubiertos > 45) & (testProcesado.metroscubiertos <= 55)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_50'
testProcesado.loc[((testProcesado.metroscubiertos > 55) & (testProcesado.metroscubiertos <= 65)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_60'
testProcesado.loc[((testProcesado.metroscubiertos > 65) & (testProcesado.metroscubiertos <= 75)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_70'
testProcesado.loc[((testProcesado.metroscubiertos > 75) & (testProcesado.metroscubiertos <= 85)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_80'
testProcesado.loc[((testProcesado.metroscubiertos > 85) & (testProcesado.metroscubiertos <= 95)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_90'
testProcesado.loc[((testProcesado.metroscubiertos > 95) & (testProcesado.metroscubiertos <= 105)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_100'
testProcesado.loc[((testProcesado.metroscubiertos > 105) & (testProcesado.metroscubiertos <= 115)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_110'
testProcesado.loc[((testProcesado.metroscubiertos > 115) & (testProcesado.metroscubiertos <= 130)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_125'
testProcesado.loc[((testProcesado.metroscubiertos > 130) & (testProcesado.metroscubiertos <= 155)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_150'
testProcesado.loc[((testProcesado.metroscubiertos > 155) & (testProcesado.metroscubiertos <= 205)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_200'
testProcesado.loc[((testProcesado.metroscubiertos > 205) & (testProcesado.metroscubiertos <= 255)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_250'
testProcesado.loc[((testProcesado.metroscubiertos > 255) & (testProcesado.metroscubiertos <= 305)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_300'
testProcesado.loc[((testProcesado.metroscubiertos > 305) & (testProcesado.metroscubiertos <= 355)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_350'
testProcesado.loc[((testProcesado.metroscubiertos > 355) & (testProcesado.metroscubiertos <= 405)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_400'
testProcesado.loc[((testProcesado.metroscubiertos > 405) & (testProcesado.metroscubiertos <= 455)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_450'
testProcesado.loc[((testProcesado.metroscubiertos > 455) & (testProcesado.metroscubiertos <= 500)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_500'
testProcesado.loc[((testProcesado.metroscubiertos > 505) & (testProcesado.metroscubiertos <= 555)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_550'
testProcesado.loc[((testProcesado.metroscubiertos > 555) & (testProcesado.metroscubiertos <= 600)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_600'
testProcesado.loc[((testProcesado.metroscubiertos > 605) & (testProcesado.metroscubiertos <= 655)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_650'
testProcesado.loc[((testProcesado.metroscubiertos > 655) & (testProcesado.metroscubiertos <= 700)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_700'
testProcesado.loc[((testProcesado.metroscubiertos > 705) & (testProcesado.metroscubiertos <= 755)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_750'
testProcesado.loc[((testProcesado.metroscubiertos > 755) & (testProcesado.metroscubiertos <= 805)), 'metroscubiertos_agrupada'] = 'metroscubiertos_agrupada_800'

testProcesado.drop('metroscubiertos', 1, inplace = True)

# AGRUPAMIENTO POR METROS TOTALES (BANDAS)
testProcesado['metrostotales_agrupada'] = 'metrostotales_agrupada_0'

testProcesado.loc[testProcesado.metrostotales <= 5, 'metrostotales_agrupada'] = 'metrostotales_agrupada_0'
testProcesado.loc[testProcesado.metrostotales > 805, 'metrostotales_agrupada'] = 'metrostotales_agrupada_1000'
testProcesado.loc[((testProcesado.metrostotales > 5) & (testProcesado.metrostotales <= 15)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_10'
testProcesado.loc[((testProcesado.metrostotales > 15) & (testProcesado.metrostotales <= 25)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_20'
testProcesado.loc[((testProcesado.metrostotales > 25) & (testProcesado.metrostotales <= 35)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_30'
testProcesado.loc[((testProcesado.metrostotales > 35) & (testProcesado.metrostotales <= 45)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_40'
testProcesado.loc[((testProcesado.metrostotales > 45) & (testProcesado.metrostotales <= 55)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_50'
testProcesado.loc[((testProcesado.metrostotales > 55) & (testProcesado.metrostotales <= 65)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_60'
testProcesado.loc[((testProcesado.metrostotales > 65) & (testProcesado.metrostotales <= 75)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_70'
testProcesado.loc[((testProcesado.metrostotales > 75) & (testProcesado.metrostotales <= 85)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_80'
testProcesado.loc[((testProcesado.metrostotales > 85) & (testProcesado.metrostotales <= 95)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_90'
testProcesado.loc[((testProcesado.metrostotales > 95) & (testProcesado.metrostotales <= 105)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_100'
testProcesado.loc[((testProcesado.metrostotales > 105) & (testProcesado.metrostotales <= 115)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_110'
testProcesado.loc[((testProcesado.metrostotales > 115) & (testProcesado.metrostotales <= 130)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_125'
testProcesado.loc[((testProcesado.metrostotales > 130) & (testProcesado.metrostotales <= 155)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_150'
testProcesado.loc[((testProcesado.metrostotales > 155) & (testProcesado.metrostotales <= 205)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_200'
testProcesado.loc[((testProcesado.metrostotales > 205) & (testProcesado.metrostotales <= 255)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_250'
testProcesado.loc[((testProcesado.metrostotales > 255) & (testProcesado.metrostotales <= 305)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_300'
testProcesado.loc[((testProcesado.metrostotales > 305) & (testProcesado.metrostotales <= 355)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_350'
testProcesado.loc[((testProcesado.metrostotales > 355) & (testProcesado.metrostotales <= 405)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_400'
testProcesado.loc[((testProcesado.metrostotales > 405) & (testProcesado.metrostotales <= 455)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_450'
testProcesado.loc[((testProcesado.metrostotales > 455) & (testProcesado.metrostotales <= 500)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_500'
testProcesado.loc[((testProcesado.metrostotales > 505) & (testProcesado.metrostotales <= 555)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_550'
testProcesado.loc[((testProcesado.metrostotales > 555) & (testProcesado.metrostotales <= 600)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_600'
testProcesado.loc[((testProcesado.metrostotales > 605) & (testProcesado.metrostotales <= 655)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_650'
testProcesado.loc[((testProcesado.metrostotales > 655) & (testProcesado.metrostotales <= 700)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_700'
testProcesado.loc[((testProcesado.metrostotales > 705) & (testProcesado.metrostotales <= 755)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_750'
testProcesado.loc[((testProcesado.metrostotales > 755) & (testProcesado.metrostotales <= 805)), 'metrostotales_agrupada'] = 'metrostotales_agrupada_800'

testProcesado.drop('metrostotales', 1, inplace = True)

In [23]:
dummies = pd.get_dummies(testProcesado['antiguedad_agrupada'], drop_first=False)
testProcesado = pd.concat([testProcesado, dummies], axis=1)

dummies = pd.get_dummies(testProcesado['metroscubiertos_agrupada'], drop_first=False)
testProcesado = pd.concat([testProcesado, dummies], axis=1)

dummies = pd.get_dummies(testProcesado['metrostotales_agrupada'], drop_first=False)
testProcesado = pd.concat([testProcesado, dummies], axis=1)

testProcesado.drop('antiguedad_agrupada', 1, inplace = True)
testProcesado.drop('metroscubiertos_agrupada', 1, inplace = True)
testProcesado.drop('metrostotales_agrupada', 1, inplace = True)

testProcesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 121 entries, id to metrostotales_agrupada_90
dtypes: category(2), float64(1), int64(15), uint8(103)
memory usage: 13.3 MB


In [24]:
#testProcesado = pd.merge(testProcesado, train_gruouped, how='left', left_on=['idzona','tipodepropiedad','habitaciones','anio'], right_on = ['idzona','tipodepropiedad','habitaciones','anio'])
#testProcesado = pd.merge(testProcesado, train_gruouped_2, how='left', left_on=['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'], right_on = ['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'])

testProcesado = pd.merge(testProcesado, train_gruouped_3, how='left', left_on=['idzona','tipodepropiedad','habitaciones','anio'], right_on = ['idzona','tipodepropiedad','habitaciones','anio'])
#testProcesado = pd.merge(testProcesado, train_gruouped_4, how='left', left_on=['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'], right_on = ['idzona','metrostotales_agrupada','metroscubiertos_agrupada','antiguedad_agrupada'])

#testProcesado['mean_propiedad'].fillna(testProcesado['mean_propiedad'].mean(), inplace = True)
#testProcesado['median_propiedad'].fillna(testProcesado['median_propiedad'].mean(), inplace = True)
#testProcesado['median_agrupado'].fillna(testProcesado['median_agrupado'].mean(), inplace = True)
#testProcesado['mean_agrupado'].fillna(testProcesado['mean_agrupado'].mean(), inplace = True)

testProcesado['mean_2016'].fillna(testProcesado['mean_2016'].mean(), inplace = True)
testProcesado['median_2016'].fillna(testProcesado['median_2016'].mean(), inplace = True)
#testProcesado['mean_2016_agrupado'].fillna(testProcesado['mean_2016_agrupado'].mean(), inplace = True)
#testProcesado['median_2016_agrupado'].fillna(testProcesado['median_2016_agrupado'].mean(), inplace = True)

In [25]:
testProcesado['mean_2016'] = testProcesado['mean_2016'].astype(int)
testProcesado['median_2016'] = testProcesado['median_2016'].astype(int)

In [26]:

#AGRUPAMIENTO POR mean_2016 (BANDAS)
testProcesado['mean_2016_agrupado'] = 'mean_2016_agrupado_0'

testProcesado.loc[testProcesado.mean_2016 <= 500000, 'mean_2016_agrupado'] = 'mean_2016_agrupado_0'
testProcesado.loc[testProcesado.mean_2016 > 11000000, 'mean_2016_agrupado'] = 'mean_2016_agrupado_22'
testProcesado.loc[((testProcesado.mean_2016 > 500000) & (testProcesado.mean_2016 <= 1000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_1'
testProcesado.loc[((testProcesado.mean_2016 > 1000000) & (testProcesado.mean_2016 <= 1500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_2'
testProcesado.loc[((testProcesado.mean_2016 > 1500000) & (testProcesado.mean_2016 <= 2000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_3'
testProcesado.loc[((testProcesado.mean_2016 > 2000000) & (testProcesado.mean_2016 <= 2500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_4'
testProcesado.loc[((testProcesado.mean_2016 > 2500000) & (testProcesado.mean_2016 <= 3000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_5'
testProcesado.loc[((testProcesado.mean_2016 > 3000000) & (testProcesado.mean_2016 <= 3500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_6'
testProcesado.loc[((testProcesado.mean_2016 > 3500000) & (testProcesado.mean_2016 <= 4000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_7'
testProcesado.loc[((testProcesado.mean_2016 > 4000000) & (testProcesado.mean_2016 <= 4500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_8'
testProcesado.loc[((testProcesado.mean_2016 > 4500000) & (testProcesado.mean_2016 <= 5000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_9'
testProcesado.loc[((testProcesado.mean_2016 > 5000000) & (testProcesado.mean_2016 <= 5500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_10'
testProcesado.loc[((testProcesado.mean_2016 > 5500000) & (testProcesado.mean_2016 <= 6000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_11'
testProcesado.loc[((testProcesado.mean_2016 > 6000000) & (testProcesado.mean_2016 <= 6500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_12'
testProcesado.loc[((testProcesado.mean_2016 > 6500000) & (testProcesado.mean_2016 <= 7000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_13'
testProcesado.loc[((testProcesado.mean_2016 > 7000000) & (testProcesado.mean_2016 <= 7500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_14'
testProcesado.loc[((testProcesado.mean_2016 > 7500000) & (testProcesado.mean_2016 <= 8000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_15'
testProcesado.loc[((testProcesado.mean_2016 > 8000000) & (testProcesado.mean_2016 <= 8500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_16'
testProcesado.loc[((testProcesado.mean_2016 > 8500000) & (testProcesado.mean_2016 <= 9000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_17'
testProcesado.loc[((testProcesado.mean_2016 > 9000000) & (testProcesado.mean_2016 <= 9500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_18'
testProcesado.loc[((testProcesado.mean_2016 > 9500000) & (testProcesado.mean_2016 <= 10000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_19'
testProcesado.loc[((testProcesado.mean_2016 > 10000000) & (testProcesado.mean_2016 <= 10500000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_20'
testProcesado.loc[((testProcesado.mean_2016 > 10500000) & (testProcesado.mean_2016 <= 11000000)), 'mean_2016_agrupado'] = 'mean_2016_agrupado_21'

# testProcesado.drop('antiguedad', 1, inplace = True)

In [27]:
#Procesamiento de columna titulo
#Parseamos las palabras seleccionadas en el DF (ver notebook de exploracion)

#Nuevo dataframe
titleFeatures = pd.DataFrame()
titleFeatures['id'] = test['id']
#Uppercase
titleFeatures['titulo'] = test['titulo'].str.upper()
#Limpiamos las columnas
titleFeatures['titulo'] = titleFeatures['titulo'].str.normalize('NFKD')\
                                                .str.encode('ascii', errors='ignore')\
                                                .str.decode('utf-8')

#Armamos las columnas para cada palabra seleccionada
#Valor de inicializacion

#Propiedad nueva
titleFeatures['NUEVA'] = 0
#Calificativo (no aporta mucho)
titleFeatures['EXCELENTE'] = 0
#Tipo de aviso
titleFeatures['VENTA'] = 0
titleFeatures['PREVENTA'] = 0
titleFeatures['RENTA'] = 0
titleFeatures['CREDITO'] = 0
titleFeatures['REMATE'] = 0
titleFeatures['OPORTUNIDAD'] = 0
#Tipo de propiedad -extra
titleFeatures['RESIDENCIA'] = 0
titleFeatures['EDIFICIO'] = 0
titleFeatures['TORRE'] = 0
titleFeatures['PISO'] = 0
titleFeatures['LOFT'] = 0
titleFeatures['FRACCIONAMIENTO'] = 0
titleFeatures['DUPLEX'] = 0
titleFeatures['COUNTRY'] = 0
titleFeatures['QUINTA'] = 0
titleFeatures['OFICINA'] = 0
titleFeatures['PENTHOUSE'] = 0
#Tipo de propiedad -extra 2
titleFeatures['REMODELADO'] = 0
titleFeatures['MODERNO'] = 0
titleFeatures['MINIMALISTA'] = 0
titleFeatures['EQUIPADA'] = 0
#Jardin
titleFeatures['JARDIN'] = 0
#Ubicacion
titleFeatures['FRENTE'] = 0
titleFeatures['ESQUINA'] = 0
titleFeatures['CENTRO'] = 0
titleFeatures['UBICACION'] = 0
#Vistas o panoramicas
titleFeatures['VISTA'] = 0
#Caracteristicas extra
titleFeatures['LUJO'] = 0
titleFeatures['BODEGA'] = 0
titleFeatures['CAMPESTRE'] = 0
titleFeatures['RANCHO'] = 0
titleFeatures['PLAYA'] = 0
titleFeatures['GOLF'] = 0
#Seguridad
titleFeatures['SEGURIDAD'] = 0

#Buscamos y agregamos 1 donde aparece la palabra
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVA', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVO', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVAS', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENE', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENAR', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVOS', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENA', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXELENTE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('BONITO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTES', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('INMEJORABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('AGRADABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('IMPECABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSAS', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESPECTACULAR', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('INCREIBLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('SUPER', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VENTA', na=False), 'VENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PREVENTA', na=False), 'PREVENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRE-VENTA', na=False), 'PREVENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RENTA', na=False), 'RENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CREDITO', na=False), 'CREDITO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HIPOTECARIO', na=False), 'CREDITO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATE', na=False), 'REMATE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATO', na=False), 'REMATE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('OPORTUNIDAD', na=False), 'OPORTUNIDAD'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIA', na=False), 'RESIDENCIA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIAL', na=False), 'RESIDENCIA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EDIFICIO', na=False), 'EDIFICIO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('TORRE', na=False), 'TORRE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PISO', na=False), 'PISO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LOFT', na=False), 'LOFT'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACCIONAMIENTO', na=False), 'FRACCIONAMIENTO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACC', na=False), 'FRACCIONAMIENTO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('DUPLEX', na=False), 'DUPLEX'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('COUNTRY', na=False), 'COUNTRY'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('QUINTA', na=False), 'QUINTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('OFICINA', na=False), 'OFICINA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PENTHOUSE', na=False), 'PENTHOUSE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADO', na=False), 'REMODELADO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADA', na=False), 'REMODELADO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNA', na=False), 'MODERNO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNO', na=False), 'MODERNO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MINIMALISTA', na=False), 'MINIMALISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EQUIPADA', na=False), 'EQUIPADA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDINES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDIN', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VERDES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ARBOLEDAS', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PARQUES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PATIO', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRENTE', na=False), 'FRENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESQUINA', na=False), 'ESQUINA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRO', na=False), 'CENTRO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRAL', na=False), 'CENTRO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICACION', na=False), 'UBICACION'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICADISIMO', na=False), 'UBICACION'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PANORAMICA', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTA', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTAS', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LUJO', na=False), 'LUJO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('BODEGA', na=False), 'BODEGA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CAMPESTRE', na=False), 'CAMPESTRE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RANCHO', na=False), 'RANCHO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PLAYA', na=False), 'PLAYA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('GOLF', na=False), 'GOLF'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('SEGURIDAD', na=False), 'SEGURIDAD'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VIGILANCIA', na=False), 'SEGURIDAD'] = 1

#Alternativamente, llenamos una sola columna como categorica, para luego encodear con LE
titleFeatures['titulo_features'] = 'NINGUNA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVA', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVO', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVAS', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENE', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENAR', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVOS', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENA', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXELENTE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('BONITO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTES', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('INMEJORABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('AGRADABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('IMPECABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSAS', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESPECTACULAR', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('INCREIBLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('SUPER', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VENTA', na=False), 'titulo_features'] = 'VENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PREVENTA', na=False), 'titulo_features'] = 'PREVENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRE-VENTA', na=False), 'titulo_features'] = 'PREVENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RENTA', na=False), 'titulo_features'] = 'RENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CREDITO', na=False), 'titulo_features'] = 'CREDITO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HIPOTECARIO', na=False), 'titulo_features'] = 'CREDITO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATE', na=False), 'titulo_features'] = 'REMATE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATO', na=False), 'titulo_features'] = 'REMATE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('OPORTUNIDAD', na=False), 'titulo_features'] = 'OPORTUNIDAD'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIA', na=False), 'titulo_features'] = 'RESIDENCIA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIAL', na=False), 'titulo_features'] = 'RESIDENCIA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EDIFICIO', na=False), 'titulo_features'] = 'EDIFICIO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('TORRE', na=False), 'titulo_features'] = 'TORRE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PISO', na=False), 'titulo_features'] = 'PISO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LOFT', na=False), 'titulo_features'] = 'LOFT'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACCIONAMIENTO', na=False), 'titulo_features'] = 'FRACCIONAMIENTO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACC', na=False), 'titulo_features'] = 'FRACCIONAMIENTO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('DUPLEX', na=False), 'titulo_features'] = 'DUPLEX'
titleFeatures.loc[titleFeatures['titulo'].str.contains('COUNTRY', na=False), 'titulo_features'] = 'COUNTRY'
titleFeatures.loc[titleFeatures['titulo'].str.contains('QUINTA', na=False), 'titulo_features'] = 'QUINTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('OFICINA', na=False), 'titulo_features'] = 'OFICINA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PENTHOUSE', na=False), 'titulo_features'] = 'PENTHOUSE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADO', na=False), 'titulo_features'] = 'REMODELADO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADA', na=False), 'titulo_features'] = 'REMODELADO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNA', na=False), 'titulo_features'] = 'MODERNO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNO', na=False), 'titulo_features'] = 'MODERNO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MINIMALISTA', na=False), 'titulo_features'] = 'MINIMALISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EQUIPADA', na=False), 'titulo_features'] = 'EQUIPADA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDINES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDIN', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VERDES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ARBOLEDAS', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PARQUES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PATIO', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRENTE', na=False), 'titulo_features'] = 'FRENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESQUINA', na=False), 'titulo_features'] = 'ESQUINA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRO', na=False), 'titulo_features'] = 'CENTRO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRAL', na=False), 'titulo_features'] = 'CENTRO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICACION', na=False), 'titulo_features'] = 'UBICACION'
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICADISIMO', na=False), 'titulo_features'] = 'UBICACION'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PANORAMICA', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTA', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTAS', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LUJO', na=False), 'titulo_features'] = 'LUJO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('BODEGA', na=False), 'titulo_features'] = 'BODEGA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CAMPESTRE', na=False), 'titulo_features'] = 'CAMPESTRE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RANCHO', na=False), 'titulo_features'] = 'RANCHO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PLAYA', na=False), 'titulo_features'] = 'PLAYA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('GOLF', na=False), 'titulo_features'] = 'GOLF'
titleFeatures.loc[titleFeatures['titulo'].str.contains('SEGURIDAD', na=False), 'titulo_features'] = 'SEGURIDAD'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VIGILANCIA', na=False), 'titulo_features'] = 'SEGURIDAD'
#La tipamos
titleFeatures['titulo_features'] = titleFeatures['titulo_features'].astype('category')

#La guardamos con label encoding
cat_features_titulo = ['titulo_features']
encoder_titulo = LabelEncoder()
encoded_titulo = titleFeatures[cat_features_titulo].apply(encoder_titulo.fit_transform)
encoded_titulo['id'] = test['id']
encoded_titulo = encoded_titulo.rename(columns={'titulo_features': 'titulo_features_enc'})

#Agregamos la columna al los DF
titleFeatures = pd.merge(titleFeatures,encoded_titulo, on='id', how='inner')
testProcesado = pd.merge(testProcesado,titleFeatures, on='id', how='inner')
testProcesado = testProcesado.drop('titulo',1)
testProcesado = testProcesado.drop('titulo_features',1)
testProcesado = testProcesado.drop('titulo_features_enc',1)

In [28]:
#Tratamiento analogo para la columna descripcion

#Procesamiento de columna descripcion
#Parseamos las palabras seleccionadas en el DF (ver notebook de exploracion)

#Nuevo dataframe
descFeatures = pd.DataFrame()
descFeatures['id'] = test['id']
#Uppercase
descFeatures['descripcion'] = test['descripcion'].str.upper()
#Limpiamos las columnas
descFeatures['descripcion'] = descFeatures['descripcion'].str.normalize('NFKD')\
                                                .str.encode('ascii', errors='ignore')\
                                                .str.decode('utf-8')

#Armamos las columnas para cada palabra seleccionada
#Valor de inicializacion

#Ya usadas en titulo
#Propiedad nueva
descFeatures['NUEVA'] = 0
#Calificativo (no aporta mucho)
descFeatures['EXCELENTE'] = 0
#Tipo de aviso
descFeatures['VENTA'] = 0
descFeatures['PREVENTA'] = 0
descFeatures['RENTA'] = 0
descFeatures['CREDITO'] = 0
descFeatures['REMATE'] = 0
descFeatures['OPORTUNIDAD'] = 0
#Tipo de propiedad -extra
descFeatures['RESIDENCIA'] = 0
descFeatures['EDIFICIO'] = 0
descFeatures['TORRE'] = 0
descFeatures['PISO'] = 0
descFeatures['LOFT'] = 0
descFeatures['FRACCIONAMIENTO'] = 0
descFeatures['DUPLEX'] = 0
descFeatures['COUNTRY'] = 0
descFeatures['QUINTA'] = 0
descFeatures['OFICINA'] = 0
descFeatures['PENTHOUSE'] = 0
#Tipo de propiedad -extra 2
descFeatures['REMODELADO'] = 0
descFeatures['MODERNO'] = 0
descFeatures['MINIMALISTA'] = 0
descFeatures['EQUIPADA'] = 0
#Jardin
descFeatures['JARDIN'] = 0
#Ubicacion
descFeatures['FRENTE'] = 0
descFeatures['ESQUINA'] = 0
descFeatures['CENTRO'] = 0
descFeatures['UBICACION'] = 0
#Vistas o panoramicas
descFeatures['VISTA'] = 0
#Caracteristicas extra
descFeatures['LUJO'] = 0
descFeatures['BODEGA'] = 0
descFeatures['CAMPESTRE'] = 0
descFeatures['RANCHO'] = 0
descFeatures['PLAYA'] = 0
descFeatures['GOLF'] = 0
#Seguridad
descFeatures['SEGURIDAD'] = 0

#Nuevas de descripcion
#Iluminacion
descFeatures['ILUMINADA'] = 0
#Ubicacion
descFeatures['TRANQUILO'] = 0
descFeatures['UNIVERSIDAD'] = 0
descFeatures['AVENIDA'] = 0
#Caracteristicas extra
descFeatures['AMENIDADES'] = 0
descFeatures['BARRA'] = 0
descFeatures['BALCON'] = 0
descFeatures['RECIBIDOR'] = 0
descFeatures['AMENIDADES'] = 0
descFeatures['DESPENSA'] = 0
descFeatures['LAVADERO'] = 0
descFeatures['REJA'] = 0
descFeatures['TERRAZA'] = 0
descFeatures['CHIMENEA'] = 0
descFeatures['CISTERNA'] = 0

#Buscamos y agregamos 1 donde aparece la palabra
descFeatures.loc[descFeatures['descripcion'].str.contains('NUEVO', na=False), 'NUEVA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTRENAR', na=False), 'NUEVA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCELENTE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSO', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AGRADABLE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSO', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESPECTACULAR', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSAS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSOS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITOS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INMEJORABLE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMATES', na=False), 'REMATE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HIPOTECARIO', na=False), 'CREDITO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSION', na=False), 'OPORTUNIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSIONISTAS', na=False), 'OPORTUNIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('RESIDENCIAL', na=False), 'RESIDENCIA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FRACCIONAMIENTO', na=False), 'FRACCIONAMIENTO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTANCIA', na=False), 'RANCHO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PISO', na=False), 'PISO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TORRE', na=False), 'TORRE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LUZ', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADA', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADO', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINACION', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PATIO', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FONDO', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JARDIN', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDE', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('GARDEN', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUE', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('NATURAL', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BOSQUES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ARBOLES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VIGILANCIA', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURIDAD', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURO', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTAS', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PANORAMICA', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTA', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICACION', na=False), 'UBICACION'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICADO', na=False), 'UBICACION'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILA', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILO', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILIDAD', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UNIVERSIDAD', na=False), 'UNIVERSIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AVENIDA', na=False), 'AVENIDA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESQUINA', na=False), 'ESQUINA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FACHADA', na=False), 'FRENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FRENTE', na=False), 'FRENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADO', na=False), 'REMODELADO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADA', na=False), 'REMODELADO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('MODERNA', na=False), 'MODERNO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCLUSIVA', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LUJO', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CALIDAD', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('MARMOL', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PORCELANATO', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JACUZZI', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIDOR', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('GOLF', na=False), 'GOLF'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BODEGA', na=False), 'BODEGA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PLAYA', na=False), 'PLAYA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AMENIDADES ', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FIESTAS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EVENTOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVANDERIA', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JUEGOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIO', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ASADOR', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CANCHAS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TENIS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BARRA', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADORA', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADOR', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BALCON', na=False), 'BALCON'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('RECIBIDOR', na=False), 'RECIBIDOR'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIBULO', na=False), 'RECIBIDOR'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CHIMENEA', na=False), 'CHIMENEA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESPENSA', na=False), 'DESPENSA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVADERO', na=False), 'LAVADERO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REJA', na=False), 'REJA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BOILER', na=False), 'CISTERNA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CISTERNA', na=False), 'CISTERNA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AZOTEA', na=False), 'TERRAZA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TERRAZA', na=False), 'TERRAZA'] = 1

#Alternativamente, llenamos una sola columna como categorica, para luego encodear con LE
descFeatures['descripcion_features'] = 'NINGUNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('NUEVO', na=False), 'descripcion_features'] = 'NUEVA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTRENAR', na=False), 'descripcion_features'] = 'NUEVA'
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCELENTE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSO', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('AGRADABLE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSO', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESPECTACULAR', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSAS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSOS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITOS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('INMEJORABLE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMATES', na=False), 'descripcion_features'] = 'REMATE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HIPOTECARIO', na=False), 'descripcion_features'] = 'CREDITO'
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSION', na=False), 'descripcion_features'] = 'OPORTUNIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSIONISTAS', na=False), 'descripcion_features'] = 'OPORTUNIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('RESIDENCIAL', na=False), 'descripcion_features'] = 'RESIDENCIA'
descFeatures.loc[descFeatures['descripcion'].str.contains('FRACCIONAMIENTO', na=False), 'descripcion_features'] = 'FRACCIONAMIENTO'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTANCIA', na=False), 'descripcion_features'] = 'RANCHO'
descFeatures.loc[descFeatures['descripcion'].str.contains('PISO', na=False), 'descripcion_features'] = 'PISO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TORRE', na=False), 'descripcion_features'] = 'TORRE'
descFeatures.loc[descFeatures['descripcion'].str.contains('LUZ', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADA', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADO', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINACION', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PATIO', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('FONDO', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('JARDIN', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDE', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('GARDEN', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUE', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('NATURAL', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('BOSQUES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('ARBOLES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VIGILANCIA', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURIDAD', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURO', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTAS', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PANORAMICA', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTA', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICACION', na=False), 'descripcion_features'] = 'UBICACION'
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICADO', na=False), 'descripcion_features'] = 'UBICACION'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILA', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILO', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILIDAD', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('UNIVERSIDAD', na=False), 'descripcion_features'] = 'UNIVERSIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('AVENIDA', na=False), 'descripcion_features'] = 'AVENIDA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESQUINA', na=False), 'descripcion_features'] = 'ESQUINA'
descFeatures.loc[descFeatures['descripcion'].str.contains('FACHADA', na=False), 'descripcion_features'] = 'FRENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('FRENTE', na=False), 'descripcion_features'] = 'FRENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADO', na=False), 'descripcion_features'] = 'REMODELADO'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADA', na=False), 'descripcion_features'] = 'REMODELADO'
descFeatures.loc[descFeatures['descripcion'].str.contains('MODERNA', na=False), 'descripcion_features'] = 'MODERNO'
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCLUSIVA', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('LUJO', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('CALIDAD', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('MARMOL', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('PORCELANATO', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('JACUZZI', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIDOR', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('GOLF', na=False), 'descripcion_features'] = 'GOLF'
descFeatures.loc[descFeatures['descripcion'].str.contains('BODEGA', na=False), 'descripcion_features'] = 'BODEGA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PLAYA', na=False), 'descripcion_features'] = 'PLAYA'
descFeatures.loc[descFeatures['descripcion'].str.contains('AMENIDADES ', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('FIESTAS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('EVENTOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVANDERIA', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('JUEGOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIO', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('ASADOR', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('CANCHAS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('TENIS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('BARRA', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADORA', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADOR', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('BALCON', na=False), 'descripcion_features'] = 'BALCON'
descFeatures.loc[descFeatures['descripcion'].str.contains('RECIBIDOR', na=False), 'descripcion_features'] = 'RECIBIDOR'
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIBULO', na=False), 'descripcion_features'] = 'RECIBIDOR'
descFeatures.loc[descFeatures['descripcion'].str.contains('CHIMENEA', na=False), 'descripcion_features'] = 'CHIMENEA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESPENSA', na=False), 'descripcion_features'] = 'DESPENSA'
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVADERO', na=False), 'descripcion_features'] = 'LAVADERO'
descFeatures.loc[descFeatures['descripcion'].str.contains('REJA', na=False), 'descripcion_features'] = 'REJA'
descFeatures.loc[descFeatures['descripcion'].str.contains('BOILER', na=False), 'descripcion_features'] = 'CISTERNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('CISTERNA', na=False), 'descripcion_features'] = 'CISTERNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('AZOTEA', na=False), 'descripcion_features'] = 'TERRAZA'
descFeatures.loc[descFeatures['descripcion'].str.contains('TERRAZA', na=False), 'descripcion_features'] = 'TERRAZA'

#La tipamos
descFeatures['descripcion_features'] = descFeatures['descripcion_features'].astype('category')

#La guardamos con label encoding
cat_features_desc = ['descripcion_features']
encoder_desc = LabelEncoder()
encoded_desc = descFeatures[cat_features_desc].apply(encoder_desc.fit_transform)
encoded_desc['id'] = test['id']
encoded_desc = encoded_desc.rename(columns={'descripcion_features': 'descripcion_features_enc'})

#Agregamos la columna al los DF
descFeatures = pd.merge(descFeatures,encoded_desc, on='id', how='inner')
testProcesado = pd.merge(testProcesado,descFeatures, on='id', how='inner')
testProcesado = testProcesado.drop('descripcion',1)
testProcesado = testProcesado.drop('descripcion_features',1)
testProcesado = testProcesado.drop('descripcion_features_enc',1)

In [29]:
#OR entre las columnas repetidas en ambos DF
#Propiedad nueva
testProcesado['NUEVA'] = testProcesado['NUEVA_x'] | testProcesado['NUEVA_y']
#Calificativo (no aporta mucho)
testProcesado['EXCELENTE'] = testProcesado['EXCELENTE_x'] | testProcesado['EXCELENTE_y']
#Tipo de aviso
testProcesado['VENTA'] = testProcesado['VENTA_x'] | testProcesado['VENTA_y'] 
testProcesado['PREVENTA'] = testProcesado['PREVENTA_x'] | testProcesado['PREVENTA_y']
testProcesado['RENTA'] = testProcesado['RENTA_x'] | testProcesado['RENTA_y'] 
testProcesado['CREDITO'] = testProcesado['CREDITO_x'] | testProcesado['CREDITO_y']
testProcesado['REMATE'] = testProcesado['REMATE_x'] | testProcesado['REMATE_y']
testProcesado['OPORTUNIDAD'] = testProcesado['OPORTUNIDAD_x'] | testProcesado['OPORTUNIDAD_y']
#Tipo de propiedad -extra
testProcesado['RESIDENCIA'] = testProcesado['RESIDENCIA_x'] | testProcesado['RESIDENCIA_y']
testProcesado['EDIFICIO'] = testProcesado['EDIFICIO_x'] | testProcesado['EDIFICIO_y'] 
testProcesado['TORRE'] = testProcesado['TORRE_x'] | testProcesado['TORRE_y']
testProcesado['PISO'] = testProcesado['PISO_x'] | testProcesado['PISO_y']
testProcesado['LOFT'] = testProcesado['LOFT_x'] | testProcesado['LOFT_y']
testProcesado['FRACCIONAMIENTO'] = testProcesado['FRACCIONAMIENTO_x'] | testProcesado['FRACCIONAMIENTO_y']
testProcesado['DUPLEX'] = testProcesado['DUPLEX_x'] | testProcesado['DUPLEX_y'] 
testProcesado['COUNTRY'] = testProcesado['COUNTRY_x'] | testProcesado['COUNTRY_y']
testProcesado['QUINTA'] = testProcesado['QUINTA_x'] | testProcesado['QUINTA_y']
testProcesado['OFICINA'] = testProcesado['OFICINA_x'] | testProcesado['OFICINA_y']
testProcesado['PENTHOUSE'] = testProcesado['PENTHOUSE_x'] | testProcesado['PENTHOUSE_y']
#Tipo de propiedad -extra 2
testProcesado['REMODELADO'] = testProcesado['REMODELADO_x'] | testProcesado['REMODELADO_y']
testProcesado['MODERNO'] = testProcesado['MODERNO_x'] | testProcesado['MODERNO_y']
testProcesado['MINIMALISTA'] = testProcesado['MINIMALISTA_x'] | testProcesado['MINIMALISTA_y']
testProcesado['EQUIPADA'] = testProcesado['EQUIPADA_x'] | testProcesado['EQUIPADA_y']
#Jardin
testProcesado['JARDIN'] = testProcesado['JARDIN_x'] | testProcesado['JARDIN_y'] 
#Ubicacion
testProcesado['FRENTE'] = testProcesado['FRENTE_x'] | testProcesado['FRENTE_y'] 
testProcesado['ESQUINA'] = testProcesado['ESQUINA_x'] | testProcesado['ESQUINA_y'] 
testProcesado['CENTRO'] = testProcesado['CENTRO_x'] | testProcesado['CENTRO_y']
testProcesado['UBICACION'] = testProcesado['UBICACION_x'] | testProcesado['UBICACION_y']
#Vistas o panoramicas
testProcesado['VISTA'] = testProcesado['VISTA_x'] | testProcesado['VISTA_y']
#Caracteristicas extra
testProcesado['LUJO'] = testProcesado['LUJO_x'] |  testProcesado['LUJO_y']
testProcesado['BODEGA'] = testProcesado['BODEGA_x'] | testProcesado['BODEGA_y']
testProcesado['CAMPESTRE'] = testProcesado['CAMPESTRE_x'] | testProcesado['CAMPESTRE_x']
testProcesado['RANCHO'] = testProcesado['RANCHO_x'] | testProcesado['RANCHO_y'] 
testProcesado['PLAYA'] = testProcesado['PLAYA_x'] | testProcesado['PLAYA_y']
testProcesado['GOLF'] = testProcesado['GOLF_x'] | testProcesado['GOLF_y']
#Seguridad
testProcesado['SEGURIDAD'] = testProcesado['SEGURIDAD_x'] | testProcesado['SEGURIDAD_y'] 

#Borramos las columnas que quedaron del merge
testProcesado.drop(list(testProcesado.filter(regex = '_x')), axis = 1, inplace = True)
testProcesado.drop(list(testProcesado.filter(regex = '_y')), axis = 1, inplace = True)

In [30]:
testProcesado.drop('idzona', axis = 1, inplace = True)
testProcesado.drop('anio', axis = 1, inplace = True)

testProcesado.drop('provincia', axis = 1, inplace = True)
testProcesado.drop('tipodepropiedad', axis = 1, inplace = True)

dummies = pd.get_dummies(testProcesado['mean_2016_agrupado'], drop_first=False)
testProcesado = pd.concat([testProcesado, dummies], axis=1)
testProcesado.drop('mean_2016_agrupado', axis = 1, inplace = True)

In [31]:
testProcesado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Columns: 189 entries, id to mean_2016_agrupado_9
dtypes: int64(66), uint8(123)
memory usage: 37.7 MB


In [32]:
#Exportamos el DF a CSV, para usar como entrada de los distintos algoritmos probados
testProcesado.to_csv('DATA/test_procesado.csv', index=False)

In [33]:
trainProcesado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Columns: 190 entries, id to mean_2016_agrupado_9
dtypes: int64(67), uint8(123)
memory usage: 152.7 MB


In [34]:
trainProcesado.columns.tolist()

['id',
 'habitaciones',
 'garages',
 'banos',
 'gimnasio',
 'usosmultiples',
 'piscina',
 'escuelascercanas',
 'centroscomercialescercanos',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 'precio',
 u'AGUASCALIENTES',
 u'BAJA CALIFORNIA NORTE',
 u'BAJA CALIFORNIA SUR',
 u'CAMPECHE',
 u'CHIAPAS',
 u'CHIHUAHUA',
 u'COAHUILA',
 u'COLIMA',
 u'DESCONOCIDA',
 u'DISTRITO FEDERAL',
 u'DURANGO',
 u'EDO. DE MAXICO',
 u'GUANAJUATO',
 u'GUERRERO',
 u'HIDALGO',
 u'JALISCO',
 u'MICHOACAN',
 u'MORELOS',
 u'NAYARIT',
 u'NUEVO LEA3N',
 u'OAXACA',
 u'PUEBLA',
 u'QUERATARO',
 u'QUINTANA ROO',
 u'SAN LUIS POTOSA',
 u'SINALOA',
 u'SONORA',
 u'TABASCO',
 u'TAMAULIPAS',
 u'TLAXCALA',
 u'VERACRUZ',
 u'YUCATAN',
 u'ZACATECAS',
 u'APARTAMENTO',
 u'BODEGA COMERCIAL',
 u'CASA',
 u'CASA EN CONDOMINIO',
 u'CASA USO DE SUELO',
 u'DEPARTAMENTO COMPARTIDO',
 u'DESCONOCIDO',
 u'HUERTA',
 u'INMUEBLES PRODUCTIVOS URBANOS',
 u'LOCAL COMERCIAL',
 u'LOCAL EN CENTRO COMERCIAL',
 u'LOTE',
 u'NAVE INDUSTRIAL',
 u'OFICINA COMERC

In [35]:
testProcesado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Columns: 189 entries, id to mean_2016_agrupado_9
dtypes: int64(66), uint8(123)
memory usage: 37.7 MB


In [36]:
testProcesado.columns.tolist()

['id',
 'habitaciones',
 'garages',
 'banos',
 'gimnasio',
 'usosmultiples',
 'piscina',
 'escuelascercanas',
 'centroscomercialescercanos',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 u'AGUASCALIENTES',
 u'BAJA CALIFORNIA NORTE',
 u'BAJA CALIFORNIA SUR',
 u'CAMPECHE',
 u'CHIAPAS',
 u'CHIHUAHUA',
 u'COAHUILA',
 u'COLIMA',
 u'DESCONOCIDA',
 u'DISTRITO FEDERAL',
 u'DURANGO',
 u'EDO. DE MAXICO',
 u'GUANAJUATO',
 u'GUERRERO',
 u'HIDALGO',
 u'JALISCO',
 u'MICHOACAN',
 u'MORELOS',
 u'NAYARIT',
 u'NUEVO LEA3N',
 u'OAXACA',
 u'PUEBLA',
 u'QUERATARO',
 u'QUINTANA ROO',
 u'SAN LUIS POTOSA',
 u'SINALOA',
 u'SONORA',
 u'TABASCO',
 u'TAMAULIPAS',
 u'TLAXCALA',
 u'VERACRUZ',
 u'YUCATAN',
 u'ZACATECAS',
 u'APARTAMENTO',
 u'BODEGA COMERCIAL',
 u'CASA',
 u'CASA EN CONDOMINIO',
 u'CASA USO DE SUELO',
 u'DEPARTAMENTO COMPARTIDO',
 u'DESCONOCIDO',
 u'HUERTA',
 u'INMUEBLES PRODUCTIVOS URBANOS',
 u'LOCAL COMERCIAL',
 u'LOCAL EN CENTRO COMERCIAL',
 u'LOTE',
 u'NAVE INDUSTRIAL',
 u'OFICINA COMERCIAL',
 u'OT